new test

In [1]:
from osgeo import gdal, gdal_array, osr
import os
import numpy as np
import sys

In [2]:
"""Total tiles for all the years = 20664"""

def reclassify_values(array):
    # Reclassify values based on specified rules
    array = np.where(array < -4000, 1, array)

    array = np.where((array >= -4000) & (array <= 99), 1, array)
    array = np.where((array >= 100) & (array <= 269), 0.75, array)
    array = np.where((array >= 270) & (array <= 439), 0.5, array)
    array = np.where((array >= 440) & (array <= 659), 0.25, array)
    array = np.where((array >= 660) & (array <= 4000), 0, array)

    array = np.where(array > 4000, 1, array)
    # fuck it, there are additional random values that fuck me up 
    # values_to_replace = [-32767, -18000] #- 32767 no burned, 32767 waterbodies, -18000 nosesabe, 23001 wtf  [23001, 32767, ]
    # for value in values_to_replace:
    #     array = np.where(array == value, 1, array) # These are the nodata, water and soil

    # array = np.where(array == 32767, np.nan, array)
    return array

# def reclassify_mask(array):
#     # Reclassify values based on specified rules
#     array = np.where((array >= -4000) & (array <= 4000), 1, array)

#     values_to_replace = [-32767, 32767, -18000]
#     for value in values_to_replace:
#         array = np.where(array == value, 255, array) # These are the nodata, water and soil
#     return array

YYYY_START = 2019
YYYY_END = 2020
input_folder = r"C:\Users\admin\Downloads\MOSEV_tiles"
output_folder = r"C:\Users\admin\Downloads\MOSEV_data"
raster_band_names = ['RdNBR'] # 'dNBR', 'preNBR', 'postNBR'
folder_list = os.listdir(input_folder)

# Dictionary to store band data for each band_name
band_data_dict = {band_name: [] for band_name in raster_band_names}
# Iterate over the year
for year in range(YYYY_START, YYYY_END + 1):
    for folder_name in folder_list[:]: #folder_list[0:1]
        folder_path = os.path.join(input_folder, folder_name)

        # create a list of the files that matches the year
        mosaic_files = []
        for file_name in os.listdir(folder_path):
            if str(year) == file_name[7:11] and file_name.endswith(".tif"):
                raster_path = os.path.join(folder_path, file_name)
                mosaic_files.append(raster_path)

        # Iterate over the specified raster band names
        for band_name in raster_band_names:
            band_index = -1
            band_data_sum = None  # Sum of band data
            num_rasters_with_data = 0  # Number of rasters with data (excluding single-band rasters)

            # Start iterating over the rasters
            for file_path in mosaic_files:
                # Open raster file
                raster_dataset = gdal.Open(file_path)
                # Find the band index for the given band_name
                for i in range(1, raster_dataset.RasterCount + 1):
                    if raster_dataset.GetRasterBand(i).GetDescription() == band_name:
                        band_index = i
                        break # if we find it, finish with the iteration

                # Read band data
                if raster_dataset.RasterCount == 1:
                    # If there's only one band, read it directly without specifying the index
                    band = raster_dataset.GetRasterBand(1)
                    # this part is only to print a message
                    if band is None:
                        # print(f"Band {band_name} is None in {file_path}. Skipping.")
                        continue
                else:
                    # If there are multiple bands, read the specified band
                    band = raster_dataset.GetRasterBand(band_index)

                band_data = band.ReadAsArray()

                # Reclassify values
                band_data = reclassify_values(band_data)

                # Check if any value in the array is greater than 1
                if np.any(band_data > 1):
                    print(f"Warning: Some values in {file_path} array exceed 1.")
                    sys.exit()

                # Accumulate band data
                if band_data_sum is None:
                    band_data_sum = band_data
                else:
                    band_data_sum += band_data

                # Increment the count if the raster has more than one band
                # if raster_dataset.RasterCount > 1:
                num_rasters_with_data += 1

                raster_dataset = None  # Close the dataset
            # Calculate average for the current band_name
            # if band_index != -1:
            average_data = band_data_sum / num_rasters_with_data

            print(f"Memory usage of average_data for {band_name}: {sys.getsizeof(average_data)} bytes")

            # Check if average_data contains only zeros
            if not np.any(average_data):
                # print(f"Average data for {band_name} is all zeros. Skipping output file creation.")
                continue

            band_data_dict[band_name].append(average_data)

            # Preapre mask with a multiplication REPLANTEALO
            # reclassify_mask(average_data)

            # Create output raster file
            output_file_name = f"{folder_name}_{year}_{band_name}_average.tif"
            output_file_path = os.path.join(output_folder, output_file_name)

            # Get information from one of the input files (assuming all have the same info)
            reference_dataset = gdal.Open(mosaic_files[0])
            driver = gdal.GetDriverByName('GTiff')
            output_dataset = driver.Create(
                output_file_path,
                reference_dataset.RasterXSize,
                reference_dataset.RasterYSize,
                1,  # Single band
                gdal_array.NumericTypeCodeToGDALTypeCode(average_data.dtype),
                options=['COMPRESS=DEFLATE', 'TILED=YES']
            )

            # Set georeferencing information
            output_dataset.SetProjection(reference_dataset.GetProjection())
            output_dataset.SetGeoTransform(reference_dataset.GetGeoTransform())

            # Write the calculated average data to the output band
            output_band = output_dataset.GetRasterBand(1)
            output_band.WriteArray(average_data)

            # Save and close the output dataset
            output_dataset.FlushCache()
            output_dataset = None  


Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usage of average_data for RdNBR: 46080128 bytes
Memory usa

In [ ]:
"""Merge the outputs"""

